In [16]:
import pandas as pd
import numpy as np
import os
import sys
import sqlite3

In [43]:
pd.set_option('display.float_format', lambda x: '{:,.0f}'.format(x).replace(',', ' '))

db_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'datalab.sqlite')

conn = sqlite3.connect(db_path)
db = conn.cursor()
query = f"""
SELECT *
FROM "data.gouv.fr.2022.clean"
"""

data = pd.read_sql_query(query, conn)

In [46]:
from scripts.preprocess_final import create_complete_pipeline, create_preprocessing_pipeline_init
from scripts.preprocess_init import drop_outliers
from sklearn.model_selection import train_test_split

In [ ]:
# data = data.drop(columns='montant')

In [47]:
data = drop_outliers(data)

In [48]:
X = data.drop(columns=['montant'])
y = data['montant']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [51]:
pipeline = create_complete_pipeline(
    cat='pred_montant',
    min=20000,
    max=50000000,
    top_n=40,
    level=2
)
X_processed = pipeline.fit_transform(X_train)

/home/julie/code/RonanB400/decp_ml/scripts/preprocess_init.py:26: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df['annee'] = pd.to_datetime(df['annee'], errors='ignore')


KeyError: 'montant'